Given completion_time and machine, check the schedule's feasibility 

In [64]:
M

array([[1, 2],
       [5, 2],
       [2, 5],
       [2, 1],
       [1, 3],
       [4, None],
       [2, 3],
       [2, 4],
       [1, 3],
       [2, 1],
       [4, 5],
       [3, None]], dtype=object)

In [65]:
['(1, 1) 2.70', '(5, 1) 0.80', '(9, 1) 0.80', 'idle 1.00', '(4, 2) 0.70', 'idle 1.40', '(10, 2) 0.50']
['(7, 1) 1.40', '(3, 1) 0.70', '(2, 2) 1.40', '(1, 2) 1.30', '(4, 1) 0.50', '(8, 1) 1.10', '(10, 1) 1.00']
['idle 1.40', '(7, 2) 2.00', 'idle 0.10', '(5, 2) 1.00', '(12, 1) 2.00', '(9, 2) 0.70']
['(6, 1) 2.50', '(11, 1) 3.00', 'idle 0.90', '(8, 2) 1.10']
['(2, 1) 1.60', 'idle 0.50', '(3, 2) 1.90', 'idle 1.50', '(11, 2) 1.40']

['(2, 1) 1.60', 'idle 0.50', '(3, 2) 1.90', 'idle 1.50', '(11, 2) 1.40']

In [66]:
instances[0]

,Job ID,Stage-1 Processing Time,Stage-2 Processing Time,Stage-1 Machines,Stage-2 Machines,Due Time
0,1,2.7,1.3,"1,2,3,4,5","2,3,4,5",5
1,2,1.6,1.4,"2,3,4,5","1,2,3,4,5",5
2,3,0.7,1.9,"1,2,3,4,5","2,3,4,5",5
3,4,0.5,0.7,"2,3,4,5","1,2,3,4,5",5
4,5,0.8,1.0,"1,2,3,4,5","2,3,4,5",5
5,6,2.5,0.0,"1,2,3,4,5",NaN,5
6,7,1.4,2.0,"2,3,4,5","2,3,4,5",5
7,8,1.1,1.1,"2,3,4,5","2,3,4,5",10
8,9,0.8,0.7,"1,2,3,4,5","2,3,4,5",10
9,10,1.0,0.5,"2,3,4,5","1,2,3,4,5",10


In [67]:
i = 0
checker(Ans[i], instances[i])

tardy: 1, [4]| makepsan: 7.9


True

In [68]:
import pandas as pd 
import numpy as np
datadir = './data'
instances = []
for i in range(5):
    name = f'instance_{i+1}.csv'
    fullpath = datadir+'/'+name
    instances.append(pd.read_csv(fullpath))

In [69]:
import joblib 
filepath = './OR_Case2_LSTratio_sol.pkl'
Ans = joblib.load(filepath)

In [70]:
'''
Job operation間一定要照順序
Machine間不可以同時多個job
（把同個machine的invoke起來）
'''

'\nJob operation間一定要照順序\nMachine間不可以同時多個job\n（把同個machine的invoke起來）\n'

In [104]:
Ans[0]

([[1, 2],
  [5, 2],
  [2, 5],
  [2, 1],
  [1, 3],
  [4, None],
  [2, 3],
  [2, 4],
  [1, 3],
  [2, 1],
  [4, 5],
  [3, None]],
 [[2.7, 4.8],
  [1.6, 3.5],
  [2.1, 4.0],
  [5.3, 6.0],
  [3.5, 4.5],
  [2.5, 2.5],
  [1.4, 3.4],
  [6.4, 7.5],
  [4.3, 7.2],
  [7.4, 7.9],
  [5.5, 6.9],
  [6.5, 6.5]])

In [110]:

def get_machine_number(df):
    mfor1 = df['Stage-1 Machines'].values.tolist()
    mfor2 = df['Stage-2 Machines'].values.tolist()
    mfor1 = [list(map(int, x.split(','))) for x in mfor1]
    mfor2 = [list(map(int, x.split(','))) for x in mfor2 if x is not np.nan]
    mfor1 = sum(mfor1, [])
    mfor2 = sum(mfor2, [])
    return max(max(mfor1), max(mfor2))



instance = instances[0]; ans = Ans[0]


def checker(ans, instance):
    
    gap = 1e-3
    n = len(instance)
    GAP = np.zeros((n))
    GAP.fill(gap)
    
    M, C_times = ans
    M, C_times = np.array(M), np.array(C_times)
    M_number = get_machine_number(instance)
    # 1. check job precedence 
    op1_ends = C_times[:, 0]
    op2_ends = C_times[:, 1]
    op1_pt = instance['Stage-1 Processing Time'].to_numpy()
    op2_pt = instance['Stage-2 Processing Time'].to_numpy()
    op_pts = np.column_stack((op1_pt, op2_pt))
    op_ends = np.column_stack((op1_ends, op2_ends))
    # print(op_ends)
    x = op1_ends + op2_pt
    # print('[Zero-index!]')
    try:
        assert np.all((op1_ends + op2_pt - op2_ends) <= GAP)
    except AssertionError:
        print('Operation precedence within the same job violated')
        print('op1 ends + op2 proc time', op1_ends + op2_pt)
        print('op2 ends (job completion times)', op2_ends)
        return False, ''
    # 2. re-construct each machine schedule
    sch = [[] for _ in range(M_number)]
    for i in range(n):
        for j in range(2):
            machine = M[i][j]
            if machine is None:
                continue
            op_end = op_ends[i][j]
            sch[machine-1].append(((i,j), op_end))

    sch = [sorted(sc, key = lambda x:x[1]) for sc in sch]
    # print(*sch, sep = '\n')
    # print(op_pts)
    # 3. check if any operations overlap
    for mi, sc in enumerate(sch):
        i = len(sc)-1
        while i > 0:
            (currop, currop_ct), (prevop, prevop_ct) = sc[i], sc[i-1]
            currop_pt = op_pts[currop[0], currop[1]]
            if currop_ct - currop_pt - prevop_ct < -gap:
                print(f'Job {[x+1 for x in currop]} and {[x+1 for x in prevop]} overlap on machine {mi+1}')
                print('prevop completion time', prevop_ct)
                print('currop processing time', currop_pt)
                print('currop completion time', currop_ct)
                
                return False, ''
            i -= 1
    # 4. compute tardy 
    dues = instance['Due Time'].to_numpy()
    tardy = list(np.where(op2_ends > dues)[0])
    # 5. compute makespan
    m_ends = [m_sche[-1][-1] for m_sche in sch]
    makespan = max(m_ends)
    return True, f'tardy: {len(tardy)}, {[x+1 for x in tardy]}| makepsan: {makespan}'

In [111]:
for i in range(5):
    passed, msg = checker(Ans[i], instances[i])
    if passed:
        print(f'Testcase {i+1} passed.')
        print(msg)
    else: print(f'Testcase {i+1} failed.')
    print('=================')

Testcase 1 passed.
tardy: 1, [4]| makepsan: 7.9
Testcase 2 passed.
tardy: 1, [7]| makepsan: 9.3
Testcase 3 passed.
tardy: 2, [2, 3]| makepsan: 13.1
Testcase 4 passed.
tardy: 6, [2, 3, 9, 10, 13, 14]| makepsan: 21.67
Testcase 5 passed.
tardy: 9, [3, 5, 6, 7, 8, 11, 15, 19, 20]| makepsan: 29.9


In [109]:
instances[1]

,Job ID,Stage-1 Processing Time,Stage-2 Processing Time,Stage-1 Machines,Stage-2 Machines,Due Time
0,1,2.7,1.5,"1,2,3,4,5","2,3,4,5",10
1,2,1.6,2.3,"1,2,3,4,5","2,3,4,5",5
2,3,1.0,2.7,"1,2,3,4,5","2,3,4,5",5
3,4,2.8,0.8,"2,3,4,5","2,3,4,5",5
4,5,0.8,1.9,"1,2,3,4,5","2,3,4,5",5
5,6,2.7,0.0,"1,2,3,4,5",NaN,5
6,7,1.4,1.5,"2,3,4,5","2,3,4,5",5
7,8,2.2,0.0,"2,3,4,5",NaN,10
8,9,0.8,1.8,"1,2,3,4,5","2,3,4,5",10
9,10,2.2,2.2,"2,3,4,5","2,3,4,5",10
